In [3]:
from time import perf_counter
import numpy as np,os
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, power_transform, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV, cross_val_predict
from sklearn.utils import shuffle
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, TomekLinks
from imblearn.ensemble import RUSBoostClassifier, BalancedRandomForestClassifier, BalancedBaggingClassifier
from imblearn.pipeline import Pipeline
import seaborn as sns,matplotlib.pyplot as plt

from library.configs import IMBS, CLFS, ENSEMBLES, CV, SCORERS
from library.utils import evaluate, read_data

In [4]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [10]:
models = {}
# for im,samp in IMBS.items():
#     for c,clf in CLFS.items():
#         models[(im,c)] = Pipeline([('samp',samp),('clf',clf)])

bases = {'nb':GaussianNB(),'dt':DecisionTreeClassifier(max_depth=20,max_features='sqrt')}
models[('UBag',"BagNB")] = BalancedBaggingClassifier(base_estimator=bases['nb'],n_estimators=20)
models[('UBag',"BagDT")] = BalancedBaggingClassifier(base_estimator=bases['dt'],n_estimators=20)
    
models.keys(),len(models)

(dict_keys([('UBag', 'BagNB'), ('UBag', 'BagDT')]), 2)

In [11]:
path = "Noisy_Eval_Bag.csv"
cols = pd.MultiIndex.from_product([IMBS.keys(),CLFS.keys(),[f.__name__ for f in SCORERS]],names=['imb','clf','metric'])
df = pd.DataFrame(index=DATASETS,columns=cols)
CV = RepeatedStratifiedKFold(n_splits=5,n_repeats=2)
#df = pd.read_csv(path,header=[0,1,2],index_col=0)

In [12]:
%%time
for it,d in enumerate(DATASETS):
    print(it)
    X,y_noisy,y_real = read_data(d,stats=True)
    for k in models:
        print(k)
        sd = perf_counter()
        r = evaluate(models[k],X,y_noisy,y_noisy,CV,SCORERS)
        for f in r:
            df.loc[d,(k[0],k[1],f)] = r[f].mean()
        print(round(perf_counter()-sd,2),[round(r[f].mean(),3) for f in r])
    print()
    df.to_csv(path)

0
activemq-5.8.0.csv noise:0.058, imb:15.847,203,3217, Shape:(3420, 65)
('UBag', 'BagNB')
0.68 [0.259, 0.325]
('UBag', 'BagDT')
0.74 [0.246, 0.297]

1
groovy-1_6_BETA_1.csv noise:0.128, imb:6.017,117,704, Shape:(821, 65)
('UBag', 'BagNB')
0.44 [0.308, 0.449]
('UBag', 'BagDT')
0.54 [0.48, 0.606]

2
activemq-5.3.0.csv noise:0.094, imb:15.669,142,2225, Shape:(2367, 65)
('UBag', 'BagNB')
0.53 [0.313, 0.39]
('UBag', 'BagDT')
0.59 [0.312, 0.337]

3
wicket-1.3.0-incubating-beta-1.csv noise:0.164, imb:4.806,288,1384, Shape:(1672, 65)
('UBag', 'BagNB')
0.55 [0.411, 0.549]
('UBag', 'BagDT')
0.62 [0.396, 0.502]

4
jruby-1.1.csv noise:0.175, imb:3.540,161,570, Shape:(731, 65)
('UBag', 'BagNB')
0.41 [0.457, 0.65]
('UBag', 'BagDT')
0.49 [0.518, 0.673]

5
jruby-1.4.0.csv noise:0.190, imb:3.890,200,778, Shape:(978, 65)
('UBag', 'BagNB')
0.47 [0.414, 0.59]
('UBag', 'BagDT')
0.52 [0.438, 0.565]

6
lucene-2.3.0.csv noise:0.204, imb:4.031,160,645, Shape:(805, 65)
('UBag', 'BagNB')
0.44 [0.34, 0.494]
('UBa

In [19]:
old = pd.read_csv("Noisy_Eval.csv",header=[0,1,2],index_col=0)
bag = pd.read_csv("Noisy_Eval_Bag.csv",header=[0,1,2],index_col=0)

In [26]:
final = pd.concat([old,bag['UBag']],axis=1)
final.shape

(32, 64)

In [28]:
final.isna().sum().sum()

0

In [29]:
final.to_csv("Noisy_Eval_final.csv")

## Analysis

In [1]:
import numpy as np
import pandas as pd,os
from scipy.stats import wilcoxon, friedmanchisquare, rankdata, trim_mean, linregress
import scikit_posthocs as sp
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
df = pd.read_csv("Noisy_Eval_final.csv",header=[0],index_col=0)

In [22]:
df.columns = pd.MultiIndex.from_tuples(map(eval,df.columns))

In [24]:
df.shape

(32, 64)

In [ ]:
df.shape

## Effect of Noise on Performance

In [ ]:
noise = pd.read_csv("Imb X Clf- Noise.csv",header=[0,1,2],index_col=0)
clean = pd.read_csv("Imb X Clf- Clean.csv",header=[0,1,2],index_col=0)
noise.shape,clean.shape

In [ ]:
wilcoxon(noise.values.reshape(-1),clean.values.reshape(-1))

In [ ]:
noise.values.mean(),clean.values.mean()

## Statistical Analysis

In [ ]:
from scipy.stats import wilcoxon, friedmanchisquare
import scikit_posthocs as sp

In [ ]:
boo = df.rename_axis('Datasets').reset_index()

In [ ]:
res = pd.melt(boo,id_vars=['Datasets'])
res.columns

In [ ]:
res

### Compare DT & RF & NB

In [ ]:
mathew =  res[res['metric']=='matthews_corrcoef'].drop(columns=['metric'])

In [ ]:
mathew = mathew[mathew['clf'].isin(['dt','rf','nb'])]
mathew.shape

In [ ]:
tmp = mathew.set_index(['Datasets','imb'])
tmp

In [ ]:
hey = tmp.pivot(columns='clf')
hey

In [ ]:
dog = hey.reset_index(drop=True)
dog.columns = dog.columns.droplevel(0)
dog

In [ ]:
friedmanchisquare(*(dog[c] for c in dog.columns))

In [ ]:
dog.mean()f

In [ ]:
pvals = pd.DataFrame(columns=dog.columns,index=dog.columns,dtype='float')
for c in dog.columns:
    for d in dog.columns:
        if c==d: continue
        pvals.loc[c,d] = float(wilcoxon(dog[c],dog[d]).pvalue)
pvals

In [ ]:
from statsmodels.stats.multitest import multipletests

In [ ]:
ps = pvals.values.reshape(-1)
idx = np.isnan(ps)
idx

In [ ]:
ps[~idx]

In [ ]:
multipletests(ps[~idx],method='fdr_by')